In [ ]:
!pip install basedosdados

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.0/204.0 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.2/135.2 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.0/106.0 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.6/25.6 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.4/108.4 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
import zipfile
from zipfile import ZipFile
import basedosdados as bd

### Setup inicial
_necessário rodar apenas uma vez_

* Download dos arquivos  
* Criação das pastas de partição (ano) em output
* Como os municípios são listados por nome, substituímos pelo id_municipio (ID Município - IBGE 7 Dígitos). Tal check cria a lista de todos os municípios que não dão match com a base de compatibilização extraída do diretório de município. Baseado nessa lista, faz-se o replace para um match 1:1

In [ ]:
# download dos dados originais
for ano in [*range(2000, 2020)]:
  if ano == 2018:
    !wget --no-check-certificate -P /content/drive/MyDrive/basedosdados/br_inpe_sisam/input https://dataserver-coids.inpe.br/queimadas/queimadas/sisam/dados_sisam-{ano}.zip

In [ ]:
# cria pastas particionadas por ano
for ano in [*range(2000, 2020)]:
  directory = '/content/drive/MyDrive/basedosdados/br_inpe_sisam/output/ano={}'.format(ano)
  if not os.path.exists(directory):
    os.makedirs(directory)

In [ ]:
# compatibilização da base entre nomes, siglas e ids
query = '''
SELECT
  sigla_uf,
  UPPER(nome_uf) AS uf_nome,
  id_municipio,
  UPPER(nome) AS municipio_nome
FROM basedosdados.br_bd_diretorios_brasil.municipio
'''

comp = bd.read_sql(query, billing_project_id='input-dados')

Downloading: 100%|██████████| 5570/5570 [00:00<00:00, 9604.05rows/s]


In [ ]:
# check de match para municipios
unmatch_list = []

for ano in [*range(2000, 2020)]:
    file = '/content/drive/MyDrive/basedosdados/br_inpe_sisam/input/dados_sisam-{}.zip'.format(ano)
    # descompacta o arquivo csv
    with ZipFile(file) as z:
      with z.open('task_9045.dados_sisam.{}.csv'.format(ano)) as f:
        df = pd.read_csv(f)
        # merge com diretorio
        df = pd.merge(df, comp, how='left', on=['uf_nome', 'municipio_nome'], indicator=True)
        # cria lista de municipios com typos
        unmatch = df.query('_merge == "left_only"')['municipio_nome'].drop_duplicates().to_list()
        # join entre anos
        unmatch_list = unmatch_list + unmatch

unmatch_list_unique = list(set(unmatch_list)) #lista conta com 28 municipios e typos para dados até 2019

### Tratamento

In [ ]:
for ano in [*range(2000, 2020)]:
  if ano == 2019:
    file = '/content/drive/MyDrive/basedosdados/br_inpe_sisam/input/dados_sisam-{}.zip'.format(ano)
    # descompacta o arquivo csv
    with ZipFile(file) as z:
      with z.open('task_9045.dados_sisam.{}.csv'.format(ano)) as f:
        df = pd.read_csv(f)
        df['municipio_nome'].replace({'ARAÇÁS':'ARAÇAS',
                                      'ATÍLIO VIVACQUA':'ATILIO VIVACQUA',
                                      'AUGUSTO SEVERO':'CAMPO GRANDE',
                                      'BIRITIBA MIRIM':'BIRITIBA-MIRIM',
                                      'FLORÍNEA':'FLORÍNIA',
                                      'IGUARACY':'IGUARACI',
                                      'ITAOCA':'ITAÓCA',
                                      'ITAPAJÉ':'ITAPAGÉ',
                                      'IUIU':'IUIÚ',
                                      'JANUÁRIO CICCO':'BOA SAÚDE',
                                      'LAURO MÜLLER':'LAURO MULLER',
                                      'MUQUÉM DO SÃO FRANCISCO':'MUQUÉM DE SÃO FRANCISCO',
                                      "OLHO D'ÁGUA DO BORGES":"OLHO-D'ÁGUA DO BORGES",
                                      'PASSA VINTE':'PASSA-VINTE',
                                      "PINGO D'ÁGUA":"PINGO-D'ÁGUA",
                                      'POXORÉU':'POXORÉO',
                                      'RESTINGA SÊCA':'RESTINGA SECA',
                                      'SANTA IZABEL DO PARÁ':'SANTA ISABEL DO PARÁ',
                                      'SÃO CRISTÓVÃO DO SUL':'SÃO CRISTOVÃO DO SUL',
                                      'SÃO LUIZ DO NORTE':'SÃO LUÍZ DO NORTE',
                                      'SÃO LUIZ DO PARAITINGA':'SÃO LUÍS DO PARAITINGA',
                                      'SÃO VICENTE FÉRRER':'SÃO VICENTE FERRER',
                                      'VESPASIANO CORRÊA':'VESPASIANO CORREA',
                                      'WESTFÁLIA':'WESTFALIA'}, inplace=True)
        df['municipio_nome'] = np.where((df['uf_nome'] == 'BAHIA') & (df['municipio_nome'] == 'SANTA TEREZINHA'), 'SANTA TERESINHA', df['municipio_nome'])
        df['municipio_nome'] = np.where((df['uf_nome'] == 'PARAÍBA') & (df['municipio_nome'] == 'QUIXABA'), 'QUIXABÁ', df['municipio_nome'])
        df = pd.merge(df, comp, how='left', on=['uf_nome', 'municipio_nome'], indicator=True)
        df['sigla_uf'] = np.where((df['uf_nome'] == 'RIO GRANDE DO SUL') & (df['municipio_nome'] == 'LAGOA MIRIM'), 'RS', df['sigla_uf'])
        df['sigla_uf'] = np.where((df['uf_nome'] == 'RIO GRANDE DO SUL') & (df['municipio_nome'] == 'LAGOA DOS PATOS'), 'RS', df['sigla_uf'])
        df['id_municipio'] = np.where((df['uf_nome'] == 'RIO GRANDE DO SUL') & (df['municipio_nome'] == 'LAGOA MIRIM'), '4300001', df['id_municipio'])
        df['id_municipio'] = np.where((df['uf_nome'] == 'RIO GRANDE DO SUL') & (df['municipio_nome'] == 'LAGOA DOS PATOS'), '4300002', df['id_municipio'])
        df.rename(columns={'precipitacao_mmdia':'precipitacao_dia',
                           'temperatura_c':'temperatura',
                           'datahora':'data_hora',
                           'umidade_relativa_percentual':'umidade_relativa',
                           'vento_direcao_grau': 'vento_direcao',
                           'vento_velocidade_ms':'vento_velocidade'}, inplace=True)
        #df = df.query('_merge == "both"')
        # df = df[['sigla_uf', 'id_municipio', 'data_hora', 'co_ppb', 'no2_ppb', 'o3_ppb', 'pm25_ugm3', 'so2_ugm3',
        #          'precipitacao_dia', 'temperatura', 'umidade_relativa', 'vento_direcao', 'vento_velocidade']]
        # df.to_csv('/content/drive/MyDrive/basedosdados/br_inpe_sisam/output/ano={}/microdados.csv'.format(ano), index=False, na_rep='')

In [ ]:
df2 = df.dropna(subset='id_municipio')

In [ ]:
df.shape

(8135120, 18)

In [ ]:
df2.shape

(4067560, 18)

In [ ]:
df2 = df.dropna(subset='id_municipio')
print(df.shape)
print(df2.shape)

(8135120, 18)
(8135120, 18)
